## Первая задача 

### Подход к решению

В начале я создала временную таблицу, чтобы немного протестирвоать запрос. 

Написание запроса разбила на несколько частей. В начале разобралась, что именно нужно и добавила несколько допущений. 
Из описания задачи были сделаны следующий выводы: 

1. Надо объеденить уроки по профессиям и посмотреть разницу внутри одного пользователя когда они заканчиваются.
2. Так как баг касается только одного пользователя и одной профессии,  мы должны собрать в начале группу всех курсов внутри одного пользователя и затем внутри этой группы собрать группы по професиям. Далее надо провести разницу во времени, использовав сортировку (нам важны только те курсы, которые были следующими по времени). 
3. Будем использовать WITH_CTE, а также оконные функции.


Возникший вопрос при решении: 
1. Первый урок среди всех уроков или только именно в профессии? То есть они начали изучать профессию data_analyst в апреле 2020б, то есть они могли изучать другие профессии ранее?
Исходя из этого вопроса было сформировано два допущения при решении: 

Допущение 1. 
Пользователь мог изучать другие профессии ранее, но если он изучал data_analysis в апреле 2020 и это был его первый урок, то он попадает в нашу кагорту. 

Допущение 2. 
При расчёте дельты мы будем смотреть разницу для выбранных пользователей за всю их историю, а не только с апреля 2020. 


Ход написания запроса был следующим: 
    
    1. Для каждого пользователя был выделен его первый урок по каждой профессии. 
    2. Были взяты пользователи, у которых по профессии data_analysis первый урок в апреле 2020. 
    3. Была взята вся история обучения для этих пользоватлей и внутри группы профессиий было взято время следующего урока
    4. Взяли разницу между временем внутри группы професии и пользователя, проранжировав время
    
    
Ниже весь запрос:     

In [1]:
"""WITH first_lessons AS (
    SELECT 
        flt.user_id,
        MIN(flt.date_created) AS first_lesson_date
    FROM 
        finished_lesson_test flt
    JOIN 
        lesson_index_test ljt ON flt.lesson_id = ljt.lesson_id
    WHERE
        ljt.profession_name = 'data-analyst' -- Выбираем только профессию "data-analyst"
    GROUP BY 
        flt.user_id
),
first_lessons_in_april AS (
    SELECT 
        user_id
    FROM 
        first_lessons
    WHERE
        DATE_FORMAT(first_lesson_date, '%Y-%m') = '2020-04' -- Выбираем только первые занятия в апреле 2020 года
), 

lesson_completion_with_delta AS (
    SELECT 
        flt.user_id,
        flt.date_created AS current_lesson_datetime,
        LEAD(flt.date_created) OVER (PARTITION BY flt.user_id, lit.profession_name ORDER BY flt.date_created) AS next_lesson_datetime,
        EXTRACT(EPOCH FROM LEAD(flt.date_created) OVER (PARTITION BY flt.user_id, lit.profession_name ORDER BY flt.date_created) - flt.date_created) AS delta_seconds,
        lit.profession_name
    FROM 
        finished_lesson_test flt
    INNER JOIN 
        lesson_index_test lit ON flt.lesson_id = lit.lesson_id
    INNER JOIN 
        first_lessons_in_april cohort ON flt.user_id = cohort.user_id
)
SELECT 
    user_id,
    current_lesson_datetime,
    next_lesson_datetime,
    delta_seconds,
    profession_name
FROM 
    lesson_completion_with_delta
WHERE 
    delta_seconds < 5;
"""

'WITH first_lessons AS (\n    SELECT \n        flt.user_id,\n        MIN(flt.date_created) AS first_lesson_date\n    FROM \n        finished_lesson_test flt\n    JOIN \n        lesson_index_test ljt ON flt.lesson_id = ljt.lesson_id\n    WHERE\n        ljt.profession_name = \'data-analyst\' -- Выбираем только профессию "data-analyst"\n    GROUP BY \n        flt.user_id\n),\nfirst_lessons_in_april AS (\n    SELECT \n        user_id\n    FROM \n        first_lessons\n    WHERE\n        DATE_FORMAT(first_lesson_date, \'%Y-%m\') = \'2020-04\' -- Выбираем только первые занятия в апреле 2020 года\n), \n\nlesson_completion_with_delta AS (\n    SELECT \n        flt.user_id,\n        flt.date_created AS current_lesson_datetime,\n        LEAD(flt.date_created) OVER (PARTITION BY flt.user_id, lit.profession_name ORDER BY flt.date_created) AS next_lesson_datetime,\n        EXTRACT(EPOCH FROM LEAD(flt.date_created) OVER (PARTITION BY flt.user_id, lit.profession_name ORDER BY flt.date_created) - flt.